# Import Statements

In [ ]:
!pip install matplotlib
!pip install pandas

In [ ]:
"""
These are our import statements
"""

import numpy as np
import matplotlib.pyplot as plt
import statistics
import random
import time  # Importing time module for tracking elapsed time
from sklearn.svm import LinearSVC  # Import the SVM classifier
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
url = 'https://raw.githubusercontent.com/jamiehadd/QuantileRKIandPerceptron/refs/heads/main/functions_QRK.py?token=GHSAT0AAAAAAC7ZSIXBTCM5DWEI7UUFJXBYZ6J44PA'
!wget --no-cache --backups=1 {url}
from functions_QRK.py import perceptronModified

# We run functions.py

In [ ]:
def QuantileRK(A, b, q, t, N, correct_labels, labels, numMislabelled, numDataPoints):
    A = np.array(A)
    m, n = A.shape
    x = np.random.rand(n)  # Initial guess for the solution
    residuals = []  # To store the residuals for plotting
    cumulative_times = []  # List to store cumulative times

    # Create a boolean mask for the labels that are in correct_labels but not in labels
    difference_mask = np.where(correct_labels == labels)  # Only retain the spots where they are equal

    # Apply the mask to filter the rows
    A_uncorrupted = A[difference_mask]
    b_uncorrupted = b[difference_mask]

    random.seed(0)  # For reproducibility

    cumulative_time = 0  # Initialize cumulative time

    # Iterate for N steps
    for j in range(N):
        start_time = time.time()  # Start time for this iteration

        condition = np.dot(A_uncorrupted, x) > b_uncorrupted  # Compute the condition

        # Count the number of elements that satisfy the condition as a percent of correct inequalities
        not_set_count = np.count_nonzero(condition) / (numDataPoints - numMislabelled) * 100

        residuals.append(not_set_count)

        # ALGORITHM IS FROM HERE DOWN
        # Randomly sample t indices from the set of m indices
        sampled_indices = np.random.choice(m, t, replace=True)

        # Pick a random index k from the m rows
        k = random.choice(np.arange(m))
        a_k = A[k]
        b_k = b[k]

        # Compute the expression for the selected index
        e = np.maximum((np.dot(x, a_k) - b_k), 0)

        # Residuals for the sampled indices
        nyet_list_of_distances = A[sampled_indices] @ x - b[sampled_indices]
        nonnegative_residuals = np.maximum(nyet_list_of_distances, 0)  # Non-negative residuals

        if e <= np.quantile(nonnegative_residuals, q):
            x = x - e * a_k
        else:
            x = x  # No update if the condition is not satisfied

        end_time = time.time() # End time for this iteration
        iteration_time = end_time - start_time
        cumulative_time += iteration_time
        cumulative_times.append(cumulative_time)  # Store the cumulative time

    return x, residuals, cumulative_times

def perceptronModified(data, labels, q, t, N, correct_labels, numMislabelled, numDataPoints):
    n, m = data.shape
    X = data.T
    Y = labels
    Y = np.diag(Y)
    X_tilda = np.matmul(Y, X)
    X_tilda = np.negative(X_tilda)
    x, residuals, cumulative_times = QuantileRK(X_tilda, np.zeros((m,)), q, t, N, correct_labels, labels, numMislabelled, numDataPoints)

    return x, residuals, cumulative_times

# Gaussian Distribution

# Synthesize the data set

In [ ]:
# Synthesize 2D data points and normalize them
numDataPoints = 50000
numFeatures = 100
data = np.random.normal(0, 1, size=(numFeatures, numDataPoints))  # Mean=0, S.D. = 1, generates a 100x50000 array
# Normalize the data
for i in range(data.shape[1]):
    data[:, i] = data[:, i] / np.linalg.norm(data[:, i])

In [ ]:
# True decision boundary separating the 2 classes of data
w_true = np.random.normal(0, 1, size=(numFeatures, 1))

# Identify points on either side of the line {x: w_true^T x = 0}
labels = np.zeros((1, data.shape[1]))
for col_ind in range(data.shape[1]):
    if np.dot(data[:, col_ind], w_true) < 0:
        labels[0, col_ind] = -1
    else:
        labels[0, col_ind] = 1

correct_labels = labels.copy()
# Create a dictionary with the data points and their labels
original_data_dict = {}
for i in range(data.shape[1]):
    key = tuple(data[:, i])
    value = labels[0, i]
    original_data_dict[key] = value

In [ ]:
'''Mislabel points'''

# Number of mislabelled points
numMislabelled = 10000

fraction_corrupted = numMislabelled / data.shape[1]
print(f"Fraction corrupted: {fraction_corrupted}")

indices = np.where(labels[0] == 1)[0]  # Find indices where labels are 1
# Mislabel the first `numMislabelled` points from the positive class
mislabelled_indices = indices[:numMislabelled]
labels[0, mislabelled_indices] = -1

# Now, create a dictionary that includes all data points with their possibly mislabelled labels
mislabelled_data_dict = {}

# Store the mislabelled data points in the dictionary
for i in range(data.shape[1]):
    key = tuple(data[:, i])  # Use tuple of the data point as the key
    value = labels[0, i]  # Corresponding label (either 1 or -1)
    mislabelled_data_dict[key] = value

mislabelled_points = {key: value for key, value in mislabelled_data_dict.items() if value == -1}
new_labels = np.array(list(original_data_dict.values()))

In [ ]:
correct_labels = correct_labels.reshape(-1) # essential line to reshape array
labels = labels.reshape(-1) # essential line to reshape array

In [ ]:
y_test = correct_labels
X_train, y_train = data.T, labels
svm_model = LinearSVC(dual=False)
svm_model.fit(X_train, y_train)
w = svm_model.coef_[0]
b = svm_model.intercept_[0]
# Manually calculate predictions for X_test using the decision function
y_pred = np.sign(np.dot(X_train, w) + b)  # Decision function: w * X + b
difference_mask = np.where(correct_labels == labels)  # Only retain the spots where they are equal
y_predUncorrupted = y_pred[difference_mask]
y_testUncorrupted = y_test[difference_mask]
# Now compare these predictions to the true labels (y_test)
accuracy = accuracy_score(y_predUncorrupted, y_testUncorrupted)
# Print results
print("Manual predictions:", y_predUncorrupted)
print("True labels (y_test):", y_testUncorrupted)
print("Accuracy:", accuracy)

# Function call and error plot

# Do one set of experiments and write up and send prof. haddock a message

In [ ]:
import numpy as np
import pandas as pd

'''
1% corruptions quantiles: 0.9, 0.95, 0.99, 1
2% corruptions quantiles: 0.9, 0.95, 0.98, 1
5% corruptions quantiles: 0.85, 0.9, 0.95, 1
10% corruptions quantiles: 0.85, 0.9, 0.95, 1
20% corruptions quantiles: 0.75, 0.8, 0.9, 1
'''

quantile_list = [0.75, 0.8, 0.9, 1]
residual_list = []

numTrials = 1
numIterations = 25000
for q in quantile_list:
  intermediateResiduals = []
  for i in range(numTrials):
      x, residuals, _ = perceptronModified(data, labels, q, numDataPoints, numIterations, correct_labels, numMislabelled, numDataPoints)
      intermediateResiduals.append(residuals)
  intermediateResiduals = np.mean(intermediateResiduals, axis = 0)
  residual_list.append(intermediateResiduals)  # Store residuals for each quantile

In [ ]:
line_styles = ['-']
plt.figure(figsize=(8, 6))
for idx, residuals in enumerate(residual_list):
    x = np.arange(len(residuals))
    plt.plot(x, residuals, label=f'{quantile_list[idx]}', linestyle=line_styles[idx], linewidth=3)

plt.xlim(0, numIterations)
plt.yticks(np.arange(0, 110, 10), fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel('Iterations', fontsize=16)
plt.ylabel('Percent of Misclassified Inequalities', fontsize=16)
plt.legend(title='Quantile', fontsize=14, title_fontsize=16)  # Increase legend title font size
plt.grid(True)
plt.show()

# Sampled Indices: Timing them

In [ ]:
sampled_indices_list = [1000, 25000, 50000]

In [ ]:
residual_list = []  # Residual times for each sampled index
residual_errors = []  # Residual errors for each sampled index
residual_dict = {}  # Dictionary to store residual_list for each t value
error_dict = {}  # Dictionary to store residual_errors for each t value
q = 1-fraction_corrupted
trials = 5
numIters = 100000

for t in sampled_indices_list:
    residual_list_i = []
    residual_errors_i = []
    for i in range(trials):
        x, residuals, residuals_time = perceptronModified(data, labels, q, t, numIters, correct_labels, numMislabelled, numDataPoints)
        residual_list_i.append(residuals_time)
        residual_errors_i.append(residuals)
    # Average the results for this sampled index across the iterations
    avg_residual_time = np.mean(residual_list_i, axis=0)
    avg_residual_error = np.mean(residual_errors_i, axis=0)

    # Append to the overall list
    residual_list.append(avg_residual_time)
    residual_errors.append(avg_residual_error)

    # Store the average residual times and errors for this t value in the corresponding dictionaries
    residual_dict[t] = avg_residual_time
    error_dict[t] = avg_residual_error

In [ ]:
line_styles = ['-', '--', '-.', ':']
markers = ['o', 's', '^', 'D']

plt.figure(figsize=(8, 6))
for idx, t in enumerate(sampled_indices_list):
    x_values = residual_dict[t]
    y_values = error_dict[t]
    line_style = line_styles[idx % len(line_styles)]
    marker = markers[idx % len(markers)]
    plt.plot(x_values, y_values, label=f'{t}', linestyle=line_style, marker=marker, linewidth=3)

plt.xlabel('Time (in seconds)', fontsize=16)
plt.ylabel('Percent of Misclassified Inequalities', fontsize=16)
plt.legend(title='Number Sampled Indices', fontsize=14, title_fontsize=16, loc='upper right')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True, which='both', axis='both')
plt.xlim(left=0)
plt.ylim(bottom=0)
# plt.savefig('time_sampled_indices_q_equals_0.80_100000_unclipped_iterations.png', dpi=300, bbox_inches='tight')  # Save with high resolution
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

for idx, t in enumerate(sampled_indices_list):
    x_values = residual_dict[t]
    y_values = error_dict[t]
    line_style = line_styles[idx % len(line_styles)]
    marker = markers[idx % len(markers)]
    plt.plot(x_values, y_values, label=f'{t}', linestyle=line_style, marker=marker, linewidth=3)

plt.xlabel('Time (in seconds)', fontsize=16)
plt.ylabel('Percent of Misclassified Inequalities', fontsize=16)
plt.legend(title='Number Sampled Indices', fontsize=14, title_fontsize=16, loc='upper right')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True, which='both', axis='both')
plt.xlim(0,1.5)
plt.ylim(bottom=0)
# plt.savefig('time_sampled_indices_q_equals_0.90_1000_clipped_iterations.png', dpi=300, bbox_inches='tight')  # Save with high resolution
plt.show()